In [42]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

divisions = ([[1,'91728'],
              [1,'82475'],
              [1,'73299'],
              [1,'62341'],
              [1,'52190'],
              [1,'45824'],
              [1,'34936'],
              [1,'30635'],
              [1,'19845'],
              [1,'12205'],
              [2,'91731']])
#athlete_list_url="https://www.athletic.net/TrackAndField/Division/Event.aspx?DivID=91731&Event=21"


In [65]:
id_key_string =     "../Athlete.aspx?AID="
school_key_string = "../School.aspx?SchoolID="

def read_div_year(Div,Div_id):
    athlete_list_url="https://www.athletic.net/TrackAndField/Division/Event.aspx?DivID="+Div_id+"&Event=21"

    response = requests.get(athlete_list_url)
    page = response.text
    soup = BeautifulSoup(page,"lxml")
    
    cols = ['ID','Name','School_ID','School','Division']
    athlete_list = pd.DataFrame(columns=cols)

    tables=soup.find_all("table")

    for i in tables:
        for row in i:
            str_row = str(row)
            id_index = str_row.find(id_key_string)
            if id_index != -1:
    # extract the athlete ID
                str_row = str_row[id_index+20:]            
                id_str = str_row[0:8]
                id_str = id_str.replace('\"',"")
                id_str = id_str.replace('>','')
                #ID = int(id_str)
    # extract the athlete Name
                name_index = str_row.find(">") + 1
                str_row = str_row[name_index:]
                name_index = str_row.find("<")
                Name = str_row[0:name_index] # save this data
    # extract the athlete school
                school_index = str_row.find(school_key_string)
                str_row = str_row[school_index+24:]
                school_index = str_row.find('\">')
                school_id = int(str_row[0:school_index]) # save this data
                str_row = str_row[school_index+2:]
                school_index = str_row.find("<")
                school_name = str_row[0:school_index] #save this data
                
                if id_str.isdigit():
                    ID = int(id_str)

                    athlete_list = athlete_list.append({'ID': ID, 'Name': Name, 'School_ID': school_id, 'School': school_name, 'Division': Div}, ignore_index=True)

    athlete_list = athlete_list.set_index(['ID'])
    return athlete_list


In [66]:
# Loop through list of division data, extracting athlete data
all_athlete_list = pd.DataFrame()
for div, div_id in divisions:
    print(div,div_id)
    new_list = read_div_year(div,div_id)
    all_athlete_list = all_athlete_list.append(new_list)
    

1 91728
1 82475
1 73299
1 62341
1 52190
1 45824
1 34936
1 30635
1 19845
1 12205
2 91731


In [68]:
all_athlete_list

,Name,School_ID,School,Division
ID,,,,
7880535,Ronna Iverson,405,Shorecrest,1
7441281,Caroline Dolan,488,Stanwood,1
12199158,Marine Wassmer,472,Arlington,1
7880539,Ami Njadoe,405,Shorecrest,1
13736186,Eloika Schemmel,405,Shorecrest,1
8645910,Kelsey Mutton,472,Arlington,1
11097622,Skye Nephew,486,Shorewood,1
11062484,Ellie Schemmel,405,Shorecrest,1
7853471,Adrienne Cordell,486,Shorewood,1


In [72]:
all_athlete_list.drop_duplicates(inplace=True)

In [73]:
all_athlete_list.to_csv('athlete_list.csv')